In [9]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
with open("../tracker_stubs/ball_detections.pki",'rb') as f:
    ball_positions = pickle.load(f)

In [1]:
ball_positions = [x.get(1,[]) for x in ball_positions]
        
# Convert the list into pandas dataframe
df_ball_positions = pd.DataFrame(ball_positions,columns = ['x1','y1','x2','y2'])

#interpolate the missing values
df_ball_positions=df_ball_positions.interpolate()
df_ball_positions = df_ball_positions.bfill()


NameError: name 'ball_positions' is not defined

In [7]:
df_ball_positions['mid_y'] = (df_ball_positions['y1']+df_ball_positions['y2'])/2
df_ball_positions['mid_y_rolling_mean'] = df_ball_positions['mid_y'].rolling(window=5,min_periods=1,center=False).mean()

In [ ]:
plt.plot(df_ball_positions['mid_y_rolling_mean'])

In [11]:
df_ball_positions['delta_y'] = df_ball_positions['mid_y_rolling_mean'].diff()

In [ ]:
plt.plot(df_ball_positions['delta_y'])

In [13]:
df_ball_positions['ball_hit']= 0


In [15]:
minimum_change_frames_for_hit = 25
for i in range(1,len(df_ball_positions)-int(minimum_change_frames_for_hit*1.2)):
    negative_position_change = df_ball_positions['delta_y'].iloc[i]>0 and df_ball_positions['delta_y'].iloc[i+1] <0
    positive_position_change = df_ball_positions['delta_y'].iloc[i]<0 and df_ball_positions['delta_y'].iloc[i+1] >0

    if negative_position_change or positive_position_change:
        change_count = 0
        for change_frame in range(i+1, i+int(minimum_change_frames_for_hit*1.2)+1):
            negative_position_change_following_frame = df_ball_positions['delta_y'].iloc[i]>0 and df_ball_positions['delta_y'].iloc[change_frame] <0
            positive_position_change_following_frame = df_ball_positions['delta_y'].iloc[i]<0 and df_ball_positions['delta_y'].iloc[change_frame] >0

            if negative_position_change and negative_position_change_following_frame:
                change_count+=1
            elif positive_position_change and positive_position_change_following_frame:
                change_count+=1
        if change_count > minimum_change_frames_for_hit-1:
            df_ball_positions['ball_hit'].iloc[i] = 1

frame_nums_with_ball_hits = df_ball_positions[df_ball_positions['ball_hit']==1].index.tolist()


C:\Users\alaga\AppData\Local\Temp\ipykernel_11016\2021909761.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_ball_positions['ball_hit'].iloc[i] =1
C:\Users\alaga\AppData\Local\Temp\ipykernel_11016\2021909761.py:17: SettingWithCopyWar

In [18]:
df_ball_positions[df_ball_positions['ball_hit']==1]

,x1,y1,x2,y2,mid_y,mid_y_rolling_mean,delta_y,ball_hit
11,775.274414,715.736572,799.397461,740.051575,727.894073,736.645013,6.548096,1
58,922.376160,234.768514,940.186279,254.438024,244.603269,241.399622,-2.032537,1
95,622.919556,747.773499,641.906769,768.585846,758.179672,774.683939,0.787241,1
131,712.657111,226.457225,730.431322,244.988118,235.722672,234.898117,-0.529108,1
182,1293.774048,739.246399,1313.961060,760.794617,750.020508,736.949454,6.358756,1


In [20]:
frame_nums_with_ball_hits = df_ball_positions[df_ball_positions['ball_hit']==1].index.tolist()

[11, 58, 95, 131, 182]